In [1]:
# 1. Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

In [2]:
# Load gold price data from 'Monthly Prices' sheet
df_gold = pd.read_excel(
    'C:/Users/Aleena Mary Abraham/OneDrive/Desktop/SCMA632_2025/Data/pink_sheet.xlsx',
    sheet_name='Monthly Prices',
    header=6  # Row with variable codes like "GOLD"
)

In [5]:
# Extract the date and gold price columns
df_gold = df_gold[['Unnamed: 0', 'GOLD']]  # 'Unnamed: 0' contains dates like 1960M01
df_gold.columns = ['Date', 'Gold_Price']

In [6]:
# Convert 'Date' from '1960M01' format to proper datetime
df_gold['Date'] = pd.to_datetime(df_gold['Date'], format='%YM%m')
df_gold.set_index('Date', inplace=True)

# Drop any missing values
df_gold = df_gold.dropna()

C:\Users\Aleena Mary Abraham\AppData\Local\Temp\ipykernel_24468\3405892041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Date'] = pd.to_datetime(df_gold['Date'], format='%YM%m')


In [7]:
# Keeping recent 20–30 years if it's too long
df_gold = df_gold[df_gold.index >= '2000-01-01']

# Preview cleaned data
print(df_gold.head())

            Gold_Price
Date                  
2000-01-01      284.32
2000-02-01      299.86
2000-03-01      286.39
2000-04-01      279.69
2000-05-01      275.19


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# 1. Split into training and test sets (last 12 months for testing)
train = df_gold.iloc[:-12]
test = df_gold.iloc[-12:]

In [ ]:
# 2. Fit SARIMA model: SARIMA(p,d,q)(P,D,Q,s)
model = sm.tsa.statespace.SARIMAX(train,
                                  order=(1, 1, 1),
                                  seasonal_order=(1, 1, 1, 12),  # Monthly seasonality
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)
results = model.fit()

In [ ]:
# 3. Forecast the next 12 months
forecast = results.get_forecast(steps=12)
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

In [ ]:
# 4. Plot forecast vs actual
plt.figure(figsize=(12, 5))
plt.plot(train.index, train['Gold_Price'], label='Training')
plt.plot(test.index, test['Gold_Price'], label='Actual')
plt.plot(forecast_mean.index, forecast_mean, label='Forecast', color='red')
plt.fill_between(forecast_mean.index,
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1],
                 color='pink', alpha=0.3)
plt.title('SARIMA Forecast of Gold Prices')
plt.xlabel('Date')
plt.ylabel('Gold Price (US$/oz)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 5. Evaluate model
rmse = np.sqrt(mean_squared_error(test, forecast_mean))
mae = mean_absolute_error(test, forecast_mean)

print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')